In [ ]:
import os
import zipfile
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Download and Extract the dataset
NUMBER_OF_EXAMPLES = 1000 # Number of images to process

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Define dictionaries for cats and dogs
cats_dir = os.path.join(train_dir, 'cats')
dogs_dir = os.path.join(train_dir, 'dogs')

# Initialize lists to store image data and labels
x_train = []
y_train = []

# Load and process the images
i = 0
for img_name in os.listdir(cats_dir):
  if i >= NUMBER_OF_EXAMPLES:
    break
    img_path = os.path.join(cats_dir, img_name)
    im = Image.open(img_path).convert("RGB")  # Open image and convert to RGB
    im_resized = im.resize((150, 150))  # Resize to 150x150 pixels
    x_train.append(np.array(im_resized))  # Convert image to NumPy array and append
    y_train.append(1)  # Label for cat
    i += 1

for img_name in os.listdir(dogs_dir):
    if i >= NUMBER_OF_EXAMPLES:
        break
    img_path = os.path.join(dogs_dir, img_name)
    im = Image.open(img_path).convert("RGB")  # Open image and convert to RGB
    im_resized = im.resize((150, 150))  # Resize to 150x150 pixels
    x_train.append(np.array(im_resized))  # Convert image to NumPy array and append
    y_train.append(0)  # Label for dog
    i += 1

x_train = np.array(x_train) / 255.0 # Normalize pixel range to [0,1]
y_train = np.array(y_train)

# Define the pre-trained ResNet50 model (without top layer)
pretrained_model = tf.keras.applications.ResNet50(
    include_top = False, # Exclude fully connected layers
    input_shape = (150, 150, 3), # Input image size
    pooling = 'avg',     # Use average pooling instead of fully connected layers
    weights = 'imagenet' # Use weights to pre-trained on ImagetNet
)

# Freeze layers of the pre-trained model
for layer in pretrained_model.layers:
  layer.trainable = False

# Add custom classification head on top of the pre-trained model
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # sigmoid for binary classification
])

# Compile the model
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Fine tune the model (unfreeze some models)
for layer in pretrained_model.layers[-20:]: # Unfreeze last 20 layers
  layer.trainable = True

# Re-compile the model w/ a lower learning rate for fine-tuning
model.compile(
    loss = 'binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Continue training the model (fine-tuning the unfrozen layers)
model.fit(x_train, y_train, epochs=5, batch_size=32, callbacks=[early_stopping])






Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - accuracy: 1.0000 - loss: 0.0856
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 1.0000 - loss: 5.0731e-08
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 1.0000 - loss: 3.3325e-08
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 1.0000 - loss: 3.2261e-08
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 1.0000 - loss: 3.2551e-08
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 1.0000 - loss: 3.1843e-08
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 1.0000 - loss: 3.0251e-08
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 1.0000 - loss: 2.9907e-08
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 1.0000 - loss: 2.8958e-08
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 1.0000 - loss: 2.7263e-08
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 1.0000 - loss: 3.9369e-09
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 1.0000 - loss: 4.1332e-09
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 146s 4s/step - accuracy: 1.0000 - loss: 4.1027e-09
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 1.0000 - loss: 4.0770e-09
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 1.0000 - loss: 3.6259e-09
